In [ ]:
# default_exp expscript

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#exports
import sys
from fastcore.all import *
from fastai.basics import *
from fastai.text.all import *

from reformer_fastai.all import *

# from reformer_fastai.reformer import LSHLM
# from reformer_fastai.data import DeterministicTwinSequence, MaskTargCallback
# from reformer_fastai.metrics import MaskedAccuracy

# from reformer_fastai.tracking import *
# from reformer_fastai.tracking import WandbCallback
# from reformer_fastai.configs import SyntheticConfig

In [ ]:
#all_slow

# Experiment Script

Experiments are run with this script by specifying:

1) The task to run, i.e. synthetic task, language modelling or translation
2) (Optionally) override default parameters for the dataloaders, models, training loop and logging

To run the training script run `run_exp` from within the `reformer_fastai` repo. For example:

```
run_exp 'synth' lr=1e-4 bs=32 
```

## Config

Sequence length of 1024 is used in the paper and training for 150 k iterations.

In [ ]:
#export

# Training
lr=1e-3

# Dataloaders
bs=128
sl=1024
train_sz=900
valid_sz=100
n_epochs=1      # we want 150 k iterations according to the paper. Adjust n_epochs according to train_szs and bs

# Model
n_hashes=4
bucket_size=64  # suggested in trax
vocab_sz=128    # specific for the synthetic task
d_model=256
n_layers=1      # specified in paper
n_heads=4
d_ff=4*d_model

attn_dropout=0.1
ff_dropout=0.1
emb_dropout=0.1

max_seq_len=sl
causal=True
use_lsh=True

Example command to run full scale experiment

run_exp 'synt' \
    --attn_dropout=0.0 \
    --ff_dropout=0.0 \
    --emb_dropout=0.0 \
    --d_ff=256 \
    --d_model=256 \
    --n_layers=1 \
    --bucket_size=64 \
    --bs=128 \
    --sl=1024 \
    --train_sz=19200000 \
    --valid_sz=10000 \
    --seed=123 \
    --wandb_group='Synthetic' \
    --wandb_tags='synthetic_task lsh lm test' \
    --wandb_name='synth_lsh_1_hash' \
    --use_lsh=True \
    --n_hashes=1 

## Data

enwik8 Data Download

In [ ]:
#export
import multiprocessing

In [ ]:
#export
def download_enwik8_data(dest='./data'):
    return untar_data('http://mattmahoney.net/dc/enwik8.zip', dest='data')
    
def get_enwik8_dataloader(data_path='/data', bs:int=8, sl:int=1024, n_workers=None, val_test_chars:int=10e6, verbose=False):
      
    if 'google.colab' in sys.modules:
        data_path = '/content' + data_path + '/enwik8'
    else:
        data_path = data_path + '/enwik8'
    
    if verbose: print('Reading data into dataframe ...')
    df = pd.DataFrame({'text':read_lines(data_path)})
    if verbose: print('done')
    
    # Do tokenization
    btt = ByteTextTokenizer(is_lm=True, add_bos=False, add_eos=False)
    if verbose: print('Tokenizing text ...')
    df['toks'] = df['text'].apply(btt)
    if verbose: print('done')
    
    # Get length of each sample and cumulative sum of lens
    df['lens'] = df['toks'].apply(len)
    df['lens_cum_sum'] = df.lens.cumsum()
    
    # Get splits, split train/valid/test based on count of tokens in each split
    train_cutoff = df.lens.sum() - val_test_chars  # keep all but 10M characters for val and test
    train_idxs = df.loc[df['lens_cum_sum'] < train_cutoff].index.values
    train_idxs = list(range(0, max(train_idxs)))

    remaining_idxs = len(df) - max(train_idxs)
    validation_idxs = list(range(max(train_idxs), max(train_idxs) + int(remaining_idxs/2)))
    test_idxs = list(range(max(validation_idxs), len(df)))

    splits = [train_idxs, validation_idxs]
     
    # Get Datasets
    if verbose: print('Setting up Datasets')
    tfms = [attrgetter("text"), btt]
    dsets = Datasets(df, [tfms], splits=splits, dl_type=LMDataLoader)
    if verbose: print('done')
    
    # Get Dataloaders
    dl_kwargs = [{'lens':df['lens'].values[train_idxs]},
                 {'val_lens':df['lens'].values[validation_idxs]}]
    if verbose: print('Setting up Dataloaders ...')
    
    n_cpus = multiprocessing.cpu_count()
    n_workers = n_cpus if n_workers is None else n_workers
    
    dls = dsets.dataloaders(bs=bs, seq_len=sl, dl_kwargs=dl_kwargs, shuffle_train=True, n_workers=n_workers)
    print('done')
    return dls

Returns dls. Half of `val_test_chars` will be assigne to validation and half to test

In [ ]:
# lm_dls = get_enwik8_dataloader(sl=65535, verbose=True)

In [ ]:
# o = lm_dls.one_batch()
# o[0].size(), o[1].size()

In [ ]:
# lm_dls.show_batch(max_n=2)

In [ ]:
# len(lm_dls.train), len(lm_dls.valid)

## Dataloaders

Twin Sequence Dataloader

In [ ]:
#export
def get_twin_sequence_dataloaders(bs:int=32, sl:int=1024, train_sz:int=500, valid_sz:int=100, seed=None):
    
    dls = DataLoaders.from_dsets(DeterministicTwinSequence(sl, train_sz, seed), 
                                 DeterministicTwinSequence(sl, valid_sz, seed), 
                                 bs=bs, shuffle=False, device='cuda')
    return dls

## Models

LSHM Model

In [ ]:
#export
def get_lshlm_model(vocab_sz:int=128, d_model:int=256, n_layers:int=1, n_heads:int=4, d_ff:int=None, 
              max_seq_len:int=64, bucket_size:int=32, n_hashes:int=4, causal:bool=True, use_lsh:bool=True,
              attn_dropout:float=0.1, ff_dropout:float=0.1, emb_dropout:float=0.1, seed=None):
    
    model = LSHLM(vocab_sz=vocab_sz, d_model=d_model, n_layers=n_layers, n_heads=n_heads, d_ff=d_ff,
                  max_seq_len=max_seq_len, bucket_size=bucket_size, n_hashes=n_hashes, causal=causal, 
                  use_lsh=use_lsh, attn_dropout=attn_dropout, ff_dropout=ff_dropout, 
                  emb_dropout=emb_dropout, seed=seed)
    return model

## Learner

Learner for Syhthetic task

In [ ]:
#export
def get_synthetic_learner(dls, model):
    
    learn = Learner(dls, model, 
                    loss_func=CrossEntropyLossFlat(ignore_index=-100), 
                    metrics=[MaskedAccuracy()]).to_fp16()
    return learn

Learner for enwik8 language modelling task

In [ ]:
#export
def get_lm_learner(dls, model, opt_func=adafactor):
    learn = Learner(dls, model, 
                    loss_func=CrossEntropyLossFlat(ignore_index=-100), 
                    opt_func=opt_func, metrics=[accuracy, perplexity, BPC()]).to_fp16()
    return learn

## Logging

In [ ]:
#export
def init_wandb(cbs:list=[], wandb_name:str='', wandb_group:str='', wandb_notes:str='', wandb_tags:str='test'):
    
    wandb_tags_ls = wandb_tags.split(' ')
    
    try:
        import wandb
        #!wandb login
    except ImportError as e:
        print(e)

    # Init wandb
    try:
        wandb_run=wandb.init(reinit=True, project="reformer-fastai", entity="fastai_community", 
               name=wandb_name, group=wandb_group, notes=wandb_notes, tags=wandb_tags_ls, config={})
        print('Weights & Biases initialised ...')
    except Exception as e:
        print(e)

    cbs.append(WandbCallback(log_model=False, log_preds=False))
    return wandb_run, cbs

## Training Script

In [ ]:
seed = 0
seed = seed if seed!=0 else None 
print(seed)

None


In [ ]:
#export
@call_parse
def run_exp(task:Param(help="Task options: 'synt','lm_base','lm_rev',lm_shared_qk, trans", type=str),
         data_path:Param(help="Path to data folder", type=str, default='./data'),
         n_epochs:Param(help="Number of epochs", type=int, default=n_epochs),
         lr:Param(help="Learning rate", type=float, default=lr),               
         bs:Param(help="Batch size", type=int, default=bs),
         sl:Param(help="Seqlence length", type=int, default=sl),
#          d_model:Param(help="Model dimension", type=int, default=d_model),
#          n_layers:Param(help="Number of model layers", type=int, default=n_layers),
#          n_heads:Param(help="Number of attention heads", type=int, default=n_heads),
#          vocab_sz:Param(help="Vocab size", type=int, default=vocab_sz),
#          d_ff:Param(help="Vocab size", type=int, default=d_ff),
#          attn_dropout:Param(help="Attention dropout rate", type=float, default=attn_dropout),
#          ff_dropout:Param(help="Attention dropout rate", type=float, default=ff_dropout),
#          emb_dropout:Param(help="Attention dropout rate", type=float, default=emb_dropout),
#          train_sz:Param(help="TwinSequence train size", type=int, default=train_sz),
#          valid_sz:Param(help="TwinSequence valid size", type=int, default=valid_sz),
#          n_hashes:Param(help="Number of LSH Attention hashes", type=int, default=n_hashes),
#          bucket_size:Param(help="LSH Attention bucket size", type=int, default=bucket_size),
#          causal:Param(help="Use causal masking", type=bool_arg, default=causal),
#          use_lsh:Param(help="Use LSH Attention", type=bool_arg, default=use_lsh),
#          max_seq_len:Param(help="Max sequence length for model embedding", type=int, default=max_seq_len),
         do_wandb_logging:Param(help="Use weights and biases logging", type=bool_arg, default=False),
         wandb_name:Param(help="wandb run name", type=str, default='my_experiment_name'),
         wandb_group:Param(help="wandb group", type=str, default='TEST'),
         wandb_notes:Param(help="wandb notes", type=str, default='My experiment notes'),
#          wandb_config:Param(help="Use wandb logging", type=bool_arg, default='my_experiment_name'),
         wandb_tags:Param(help="wandb tags, add tags in a single string, space separated", type=str, default='test'),
         save_model:Param(help="Save model locally in /models", type=bool_arg, default=False),
         cuda_id:Param(help="Which cuda device to use", type=int, default=0),
         seed:Param(help="Set seed for reproducibiltiy, passing anything except 0 will use fastai's set_seed", type=int, default=0),
#          verbose:Param(help="Print script logs", type=bool_arg, default=False)
        ):

    """Task options: 'synt','lm_base','lm_rev',lm_shared_qk, trans"""
    
    # Callbacks used for training
    cbs = []               
    
    #random seeds
    if seed!=0:
        set_seed(seed, reproducible=True)  # this  sets `torch.cudnn.backends ++`
    else: 
        seed = None   # this is passed to LSH and data generator. They expect None or int
      
    # Set which GPU to run the script on
    torch.cuda.set_device(cuda_id)
    
    if task == 'synt':
        print('Getting dataloaders ...')
        dls = get_twin_sequence_dataloaders(bs=bs, sl=sl, train_sz=train_sz, valid_sz=valid_sz, seed=seed)
        print('done!')
        
        print('Getting model ...')
        config = SyntheticConfig(warn=False, verbose=verbose, **locals())
        config.save(wandb_name, add_tstmp=True)
        model = LSHLM.from_config(config)
        print('done!')
        
        print('Getting learner ...')
        learn = get_synthetic_learner(dls, model)
        print('done!')
        
        # Set up Weights & Biases logging, if needed
        if do_wandb_logging: 
            wandb_run, cbs = init_wandb(cbs, wandb_name=wandb_name, wandb_group=wandb_group,
                                        wandb_notes=wandb_notes, wandb_tags=wandb_tags)
        
        # Append training callbacks needed
        cbs.append(MaskTargCallback())
        
        # Start training
        print('Starting training...')
        learn.fit_one_cycle(n_epochs, lr, cbs=cbs)
        print('done!')
        
        # Close wandb logging for this run
        if do_wandb_logging: wandb_run.finish()
        
        # Save model weights if needed, saved in /models relative to where script is run
        if save_model:
            now = time.strftime("_%d_%m_%Y_%H:%M", time.gmtime())
            learn.save(f'{task}_{wandb_name}_{now}')
    
    elif 'lm' in task:
        
        if task == 'lm_base':
            config = TransformerLMConfigEnwik8(warn=False, verbose=False, **locals())
            print('Getting model ...')
            model = TransformerLM.from_config(config)
            print('done!')
        elif task == 'lm_rev':
            config = ReversibleLMConfigEnwik8(warn=False, verbose=False, **locals())
            print('Getting model ...')
            model = ReversibleLM.from_config(config)
            print('done!')
        elif task == 'lm_shared_qk':
            config = TransformerLMConfigEnwik8(warn=False, verbose=False, **locals())
            config['shared_qk'] = True
            print('Getting model ...')
            model = TransformerLM.from_config(config)
            print('done!')
        
        config.save(wandb_name, add_tstmp=True)

        print('Checking data')
        download_enwik8_data(dest='./data')
        print('done')
        
        print('Getting dataloaders ...')
        dls = get_enwik8_dataloader(data_path=data_path, bs=bs, sl=sl,verbose=True)
        print('done')
        
        print('Getting learner ...')
        learn = get_lm_learner(dls, model, opt_func=adafactor)
        print('done!')
        
        # Set up Weights & Biases logging, if needed
        if do_wandb_logging: 
            wandb_run, cbs = init_wandb(cbs, wandb_name=wandb_name, wandb_group=wandb_group,
                                        wandb_notes=wandb_notes, wandb_tags=wandb_tags)
        
        # Append training callbacks needed
#         cbs.append()
        
        # Start training
        print('Starting training...')
        learn.fit_one_cycle(n_epochs, lr, cbs=cbs)
        print('done!')
        
        # Close wandb logging for this run
        if do_wandb_logging: wandb_run.finish()
        
        # Save model weights if needed, saved in /models relative to where script is run
        if save_model:
            now = time.strftime("_%d_%m_%Y_%H:%M", time.gmtime())
            learn.save(f'{task}_{wandb_name}_{now}')
        
    elif task == 'test_cfg':
        print('Locals ', locals())
        print()
        config = SyntheticConfig(verbouse=True, **locals())
        print(config)
        config.save('test')
        config2 = SyntheticConfig.from_file('test')
        print(config2)
        
    elif task == 'test':
        print('testing testing :)')
        
    else:
        print('No task run')

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_attention.ipynb.
Converted 03_transformer.ipynb.
Converted 04_reformer.ipynb.
Converted 05_tokenizers.ipynb.
Converted 06_data.ipynb.
Converted 07_metrics.ipynb.
Converted 08_optimizers.ipynb.
Converted 09_tracking.ipynb.
Converted 10_experiment.synthetic-task.ipynb.
Converted 10a_experiment.synthetic-task-comparison.ipynb.
Converted 10b_experiment.synthetic-task-minimal.ipynb.
Converted 10c_experiment.synthetic-task-analysis.ipynb.
Converted 11_experiment.enwik8_baseline.ipynb.
Converted 12_experiment.enwik8_sharedQK.ipynb.
Converted 13_experiment.enwik8_reversible.ipynb.
Converted 20_experiment-script.ipynb.
Converted 21_experiment-configs.ipynb.
Converted 50_exploration.LSH.ipynb.
Converted index.ipynb.
Converted reproducibility.report_1_reproducibility_summary.ipynb.
